In [3]:
import numpy as np;

In [6]:
a = np.arange(12)

In [ ]:
a = np.aran

In [9]:
a

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [16]:
np.mean(a)

2.75

In [15]:
np.mean(a[a>5])

5.5

In [12]:
a = a/2

In [13]:
a

array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5])

In [26]:
for e,i in enumerate(a, start=4):
    print(e, i)

4 0.0
5 0.5
6 1.0
7 1.5
8 2.0
9 2.5
10 3.0
11 3.5
12 4.0
13 4.5
14 5.0
15 5.5


In [6]:
file_list = ['r1.tif', 'r2.tif', 'r3.tif']

In [7]:
[print(f, other) for f,other in zip(file_list, [0,1,2,3])]

r1.tif 0
r2.tif 1
r3.tif 2


[None, None, None]

In [51]:
import os
os.getcwd()

'/mnt/d/work/DeepLearningSatelliteImage/1_GettingStarted'

In [3]:
import rasterio
import glob

In [4]:
def write_image(input, output, band):
    with rasterio.open(input) as src_dataset:
        with rasterio.open('example_TCI_10m.jp2', driver='JP2OpenJPEG') as dataset:
            array = dataset.read(band)
        # Get a copy of the source dataset's profile. Thus our
        # destination dataset will have the same dimensions,
        # number of bands, data type, and georeferencing as the
        # source dataset.
        kwds = src_dataset.profile

        # Change the format driver for the destination dataset to
        # 'GTiff', short for GeoTIFF.
        kwds['driver'] = 'GTiff'

        # Add GeoTIFF-specific keyword arguments.
        kwds['tiled'] = True
        kwds['blockxsize'] = 256
        kwds['blockysize'] = 256
        kwds['photometric'] = 'YCBCR'
        kwds['compress'] = 'JPEG'

        with rasterio.open(output, 'w', **kwds) as dst_dataset:
            # Write data to the destination dataset.
            dst_dataset.write(array.astype(rasterio.uint8), band)

In [8]:
[write_image('example_TCI_10m.jp2',f, band) for f,band in zip(file_list, [1, 2, 3])]

[None, None, None]

In [246]:
def merge_bands(bands_tiff_list, output_tiff_name):
    with rasterio.open(bands_tiff_list[0]) as band:
        out_meta = band.meta

    out_meta.update(count = len(bands_tiff_list))

    with rasterio.open(output_tiff_name, 'w', **out_meta) as out_tiff:
        for i, band_tiff in enumerate(bands_tiff_list, start=1):
            print('writing {} @index:{} ...'.format(band_tiff, i))
            with rasterio.open(band_tiff) as band:
                print('band profile: {}'.format(band.profile))
                out_tiff.write_band(i, band.read(i))

    print('done writing {}'.format(output_tiff_name))

In [247]:
merge_bands(file_list, 'stack.tif')

writing r1.tif @index:1 ...
band profile: {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 3, 'crs': CRS({'init': 'epsg:32719'}), 'transform': (600000.0, 10.0, 0.0, 8000020.0, 0.0, -10.0), 'affine': Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 8000020.0), 'blockxsize': 256, 'blockysize': 256, 'compress': 'jpeg', 'interleave': 'pixel', 'photometric': 'ycbcr', 'tiled': True}
writing r2.tif @index:2 ...
band profile: {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 3, 'crs': CRS({'init': 'epsg:32719'}), 'transform': (600000.0, 10.0, 0.0, 8000020.0, 0.0, -10.0), 'affine': Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 8000020.0), 'blockxsize': 256, 'blockysize': 256, 'compress': 'jpeg', 'interleave': 'pixel', 'photometric': 'ycbcr', 'tiled': True}
writing r3.tif @index:3 ...
band profile: {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 3, 'crs': CR

In [9]:
def print_meta(input_tiff):
    with rasterio.open(input_tiff) as tiff:
        print(tiff.meta)

In [10]:
print_meta('stack.tif')

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 3, 'crs': CRS.from_epsg(32719), 'transform': Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 8000020.0)}


In [11]:
from rasterio.warp import calculate_default_transform
from rasterio.warp import reproject, Resampling

def reproject_tif_to_crs(tif_file, dest_crs, out_tif_file):
    with rasterio.open(tif_file) as src_tif:
        trans, width, height = calculate_default_transform(
        src_tif.crs, dest_crs, src_tif.width, src_tif.height,
        *src_tif.bounds)

        meta_wargs = src_tif.meta.copy()
        meta_wargs.update({
            'crs' : dest_crs,
            'transform' : trans,
            'width' : width,
            'height' : height
        })

        print ('reproject tif:{}\n from:{}\n to:{}\n'
               .format(tif_file, src_tif.meta, meta_wargs))
        print ('writing reprojected tif to:{}'
               .format(out_tif_file))

        with rasterio.open(out_tif_file, 'w',
                       **meta_wargs) as dest_tif:
            for band_idx in range(1, src_tif.count + 1):
                reproject(
                    source=rasterio.band(src_tif, band_idx),
                    destination=rasterio.band(dest_tif, band_idx),
                    src_transform=src_tif.transform,
                    src_crs=src_tif.crs,
                    dst_transform=trans,
                    dst_crs=dest_crs,
                    resampling=Resampling.nearest)

In [250]:
reproject_tif_to_crs('stack.tif', 'EPSG:4326',
                     'reprojected_stack.tif')

reproject tif:stack.tif
 from:{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 3, 'crs': CRS({'init': 'epsg:32719'}), 'transform': (600000.0, 10.0, 0.0, 8000020.0, 0.0, -10.0), 'affine': Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 8000020.0)}
 to:{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 11325, 'height': 10799, 'count': 3, 'crs': 'EPSG:4326', 'transform': Affine(9.261262692878677e-05, 0.0, -68.05495401811122,
       0.0, -9.261262692878677e-05, -18.078345401611326), 'affine': Affine(10.0, 0.0, 600000.0,
       0.0, -10.0, 8000020.0)}

writing reprojected tif to:reprojected_stack.tif


/mnt/d/anaconda3/envs/liveproject/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  


In [251]:
print_meta('reprojected_stack.tif')

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 11325, 'height': 10799, 'count': 3, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-68.05495401811122, 9.261262692878677e-05, 0.0, -18.078345401611326, 0.0, -9.261262692878677e-05), 'affine': Affine(9.261262692878677e-05, 0.0, -68.05495401811122,
       0.0, -9.261262692878677e-05, -18.078345401611326)}


In [88]:
import rasterio
print("rasterio's version is: " + rasterio.__version__)

rasterio's version is: 1.1.0


#### Change image resolution example

In [22]:
import numpy as np
import rasterio
from rasterio import Affine as A
from rasterio.warp import reproject, Resampling

# change resolution by scale
def resample_tif_resolution(tif_file, scale, out_tif_file):
    with rasterio.open(tif_file) as src_tif:
        src_transform = src_tif.transform
        dst_transform = src_transform

        data = src_tif.read()
        kwargs = src_tif.meta
        kwargs['width'] = src_tif.width//scale
        kwargs['height'] = src_tif.height//scale

        with rasterio.open(out_tif_file, 'w', **kwargs) as dst:
            for i, band in enumerate(data, 1):
                dest = np.zeros_like(band)

                reproject(band,
                    dest,
                    src_transform=src_transform,
                    src_crs=src_tif.crs,
                    dst_transform=dst_transform,
                    dst_crs=src_tif.crs,
                    resampling=Resampling.bilinear)

                dst.write(dest, indexes=i)

In [29]:
resample_tif_resolution('reprojected_stack.tif', 5, 'reprojected_stack_5x.tif')

In [30]:
resample_tif_resolution('reprojected_stack_5x.tif', 0.9, 'reprojected_stack_5x_09x.tif')

#### Simple zoom and compress

In [74]:
from rasterio import Affine

# simple zoom in/out by scale
def zoom_scale_tif(tif_file, scale, out_tif_file):
    with rasterio.open(tif_file) as src_tif:
        src_transform = src_tif.transform
        dst_transform = (src_transform
                         *A.translation(-src_tif.width / scale,
                                        -src_tif.height / scale)
                         *A.scale(scale))

        data = src_tif.read()
        kwargs = src_tif.meta
        kwargs['transform'] = dst_transform
        kwargs['compress'] ='lzw'

        with rasterio.open(out_tif_file, 'w', **kwargs) as dst:
            for i, band in enumerate(data, 1):
                dest = np.zeros_like(band)

                reproject(band,
                    dest,
                    src_transform=src_transform,
                    src_crs=src_tif.crs,
                    dst_transform=dst_transform,
                    dst_crs=src_tif.crs,
                    resampling=Resampling.nearest)

                dst.write(dest, indexes=i)

In [75]:
zoom_scale_tif('reprojected_stack_5x_09x.tif', 2.0, 'reprojected_stack_5x_09x_2z0.tif' )

#### Save partial image using `rasterio.windows`

In [87]:
from rasterio.windows import Window
from rasterio.windows import get_data_window

with rasterio.open('reprojected_stack_5x_09x.tif') as src:
    b, g, r = (src.read(k, window=Window(100, 100, 612, 356)) for k in (1, 2, 3))

with rasterio.open(
        'reprojected_stack_5x_09x_win.tif', 'w',
        driver='GTiff', width=512, height=256, count=3,
        dtype=r.dtype) as dst:
    for k, arr in [(1, b), (2, g), (3, r)]:
        dst.write(arr, indexes=k, window=Window(0, 0, 512, 256))

#### TensorFlow & Keras: image preprocessing example

In [91]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def get_seq_model(img_size=(28,28)):
    return keras.Sequential([
      layers.Flatten(input_shape=img_size),
      layers.Dense(128, activation='relu'),
      layers.Dropout(0.2),
      layers.Dense(10, activation='softmax')
    ])

In [92]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#### The resize functions (`tf.image.resize`)

In [93]:
def resize_np_img(np_2d, new_img_size=(64, 64)):
    # add channel
    np_2d_1_ch = np.reshape(np_2d, (*np_2d.shape, 1))
    # resize using TF
    img = tf.image.resize(np_2d_1_ch, new_img_size, method='gaussian')
    # converto to numpy
    np_2d_1_ch_resized = tf.keras.preprocessing.image.img_to_array(img)
    return np.reshape(np_2d_1_ch_resized, new_img_size)

def resize_np_images(images, new_size=(64,64)):
    print('resize_np_images to {}:{}'.format(*new_size))
    resized_imgs = np.empty((images.shape[0], *new_size), images.dtype)
    for idx,img in enumerate(images):
        resized_imgs[idx] = resize_np_img(img, new_size)

    return resized_imgs

In [94]:
new_shape=(16,16)
x_train=resize_np_images(x_train, new_shape)
x_test=resize_np_images(x_test, new_shape)

resize_np_images to 16:16
resize_np_images to 16:16


In [95]:
model16x16=get_seq_model(new_shape)
model16x16.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 34,186
Trainable params: 34,186
Non-trainable params: 0
_________________________________________________________________


In [96]:
model16x16.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

In [97]:
model16x16.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 6s 115us/sample - loss: 1.7285 - accuracy: 0.7657 - val_loss: 1.6320 - val_accuracy: 0.8400
Epoch 2/10
48000/48000 [==============================] - 4s 76us/sample - loss: 1.6309 - accuracy: 0.8387 - val_loss: 1.6157 - val_accuracy: 0.8497
Epoch 3/10
48000/48000 [==============================] - 4s 84us/sample - loss: 1.6168 - accuracy: 0.8495 - val_loss: 1.6057 - val_accuracy: 0.8587
Epoch 4/10
48000/48000 [==============================] - 4s 79us/sample - loss: 1.6079 - accuracy: 0.8576 - val_loss: 1.6002 - val_accuracy: 0.8639
Epoch 5/10
48000/48000 [==============================] - 4s 89us/sample - loss: 1.6011 - accuracy: 0.8638 - val_loss: 1.5963 - val_accuracy: 0.8671
Epoch 6/10
48000/48000 [==============================] - 4s 76us/sample - loss: 1.5975 - accuracy: 0.8669 - val_loss: 1.5927 - val_accuracy: 0.8705
Epoch 7/10
48000/48000 [==============================]

In [98]:
model16x16.evaluate(x_test, y_test, verbose=0)

[1.4992268077850341, 0.9664]